In this notebook, we will clean and transform the permit data we already cleaned as well as clean and transform data about each metro station, downloaded directly from Los Angeles Open Data.

In [1]:
import numpy as np
import pandas as pd
import googlemaps
from sodapy import Socrata
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
gmaps = googlemaps.Client(key='key')

In [3]:
metro_lines = pd.read_csv('Metro_Rail_Lines_Stops.csv')

permit_data = pd.read_excel('transformed_city_permit_data.xlsx')

In [4]:
permit_data.head()

,Unnamed: 0,pcis_permit,work_description,census_tract,permit_category,initiating_office,valuation,of_residential_dwelling_units,address_start,street_direction,street_name,street_suffix,zip_code,issue_year,issue_month,issue_day,issue_date,permit_sub_type,of_stories
0,0,20010-20000-01945,"PROPOSED (51'-6"" X 101'-6"") IRREGULAR SHAPED S...",1393.01000,Plan Check,VAN NUYS,800000.00000,1,5826,N,MELVIN,AVE,91356,2020,10,29,2020-10-29,1 or 2 Family Dwelling,2.00000
1,1,17010-20000-00961,"NEW 24'-0"" x 45'-0"" TWO STORY SFD W/ ATTACHED ...",1871.02000,Plan Check,VAN NUYS,178000.00000,1,3133,W,LACLEDE,AVE,90039,2018,5,2,2018-05-02,1 or 2 Family Dwelling,2.00000
2,2,19010-20000-05164,TWO STORY SFD WITH ATTACHED 3 CAR GARAGE _STAN...,1082.02000,Plan Check,VAN NUYS,604000.00000,1,20318,W,WINDSOR,LANE,91326,2020,3,3,2020-03-03,1 or 2 Family Dwelling,2.00000
3,3,16010-30000-04199,New SFD with attached garage and recroom.,2721.00000,Plan Check,WEST LA,1055000.00000,1,3715,S,WASATCH,AVE,90066,2017,5,22,2017-05-22,1 or 2 Family Dwelling,2.00000
4,4,16010-20000-01617,PROPOSED IRR TWO STORY SFD WITH ATTACHED TWO C...,2719.02000,Plan Check,VAN NUYS,462000.00000,1,3564,S,GRAND VIEW,BLVD,90066,2017,3,7,2017-03-07,1 or 2 Family Dwelling,2.00000


In [5]:
# create a copy of the df to make changes to
permit_data_transformed = permit_data.copy()

# ensure all parts of the address are strings
permit_data_transformed['address_start'] = permit_data_transformed['address_start'].astype('str')
permit_data_transformed['street_direction'] = permit_data_transformed['street_direction'].astype('str')
permit_data_transformed['street_name'] = permit_data_transformed['street_name'].astype('str')
permit_data_transformed['street_suffix'] = permit_data_transformed['street_suffix'].astype('str')
permit_data_transformed['zip_code'] = permit_data_transformed['zip_code'].astype('str')
permit_data_transformed['city'] = 'LOS ANGELES, CA'

In [6]:
permit_data_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23776 entries, 0 to 23775
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Unnamed: 0                     23776 non-null  int64         
 1   pcis_permit                    23776 non-null  object        
 2   work_description               23774 non-null  object        
 3   census_tract                   23774 non-null  float64       
 4   permit_category                23776 non-null  object        
 5   initiating_office              23776 non-null  object        
 6   valuation                      23767 non-null  float64       
 7   of_residential_dwelling_units  23776 non-null  int64         
 8   address_start                  23776 non-null  object        
 9   street_direction               23776 non-null  object        
 10  street_name                    23776 non-null  object        
 11  street_suffix  

In [7]:
# combine address into single column
permit_data_transformed['address_full'] = permit_data_transformed['address_start'].str.cat(permit_data_transformed[[\
                                                        'street_direction', 'street_name',                                                          
                                                        'street_suffix','city', 'zip_code']], sep = ' ')
                                                                                
permit_data_transformed['address_full']
permit_data_transformed.head()
permit_data_transformed.to_csv(r'permit_data_transformed.csv')

In [8]:
# add column to differentiate permits from metro stops, prepare for entry of long and lat
permit_data_transformed['type'] = 'Permit'
permit_data_transformed['lat'] = ''
permit_data_transformed['long'] = ''


In [9]:
# use gmaps api to locate longitude and latitude of each permit address

for x in range(len(permit_data_transformed)):
    try:
        geocode_result = gmaps.geocode(permit_data_transformed['address_full'][x])
        permit_data_transformed.loc[x, 'lat'] = geocode_result[0]['geometry']['location']['lat']
        permit_data_transformed.loc[x, 'long'] = geocode_result[0]['geometry']['location']['lng']
        
    except IndexError:
        print("Address was wrong...")
    except Exception as e:
        print("Unexpected error occurred.", e )

permit_data_transformed.head()

In [10]:
permit_data_transformed.tail()


,Unnamed: 0,pcis_permit,work_description,census_tract,permit_category,initiating_office,valuation,of_residential_dwelling_units,address_start,street_direction,street_name,street_suffix,zip_code,issue_year,issue_month,issue_day,issue_date,permit_sub_type,of_stories,city,address_full,type,lat,long
23771,23771,13010-10000-00711,"New 5-story, 703 unit apartment over 1 level s...",2756.02000,Plan Check,METRO,155225082.00000,703,12411,W,FIELDING,CIR,90094,2014,4,3,2014-04-03,Apartment,5.00000,"LOS ANGELES, CA","12411 W FIELDING CIR LOS ANGELES, CA 90094",Permit,,
23772,23772,15010-10000-00347,Mixed use builidng (Apartment/retail/parking g...,2077.10000,Plan Check,METRO,156989672.00000,735,1000,W,8TH,ST,90017,2015,11,12,2015-11-12,Commercial,56.00000,"LOS ANGELES, CA","1000 W 8TH ST LOS ANGELES, CA 90017",Permit,,
23773,23773,18010-10000-03720,New 64-story mixed-use Apartment and Retail bu...,2077.10000,Plan Check,METRO,259597503.00000,785,960,W,7TH,ST,90017,2020,7,17,2020-07-17,Commercial,64.00000,"LOS ANGELES, CA","960 W 7TH ST LOS ANGELES, CA 90017",Permit,,
23774,23774,16010-10000-04628,NEW 910-UNIT MIXED-USE BUILDING (CORE AND SHEL...,2199.02000,Plan Check,METRO,242200327.00000,910,3323,S,LA CIENEGA,BLVD,90016,2018,6,28,2018-06-28,Commercial,7.00000,"LOS ANGELES, CA","3323 S LA CIENEGA BLVD LOS ANGELES, CA 90016",Permit,,
23775,23775,15010-10000-02603,NEW SEVEN STORY MIXED USE BUILDING WITH TWO LE...,2080.00000,Plan Check,METRO,255000000.00000,1150,1000,W,TEMPLE,ST,90012,2018,6,27,2018-06-27,Apartment,7.00000,"LOS ANGELES, CA","1000 W TEMPLE ST LOS ANGELES, CA 90012",Permit,,


In [11]:
# prepare metro table to match permit table
metro_lines['type'] = 'Metro'
metro_lines.head()

,X,Y,OBJECTID,MetroLine,Station,StopNumber,TOOLTIP,NLA_URL,type
0,-118.19292,33.76807,1,Blue Line,Downtown Long Beach Station,80101,Stop: Downtown Long Beach Station\nStop No: 80...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
1,-118.19370,33.77226,2,Blue Line,Pacific Ave Station,80102,Stop: Pacific Ave Station\nStop No: 80102\nBlu...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
2,-118.18938,33.78183,3,Blue Line,Anaheim Street Station,80105,Stop: Anaheim Street Station\nStop No: 80105\n...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
3,-118.18938,33.78909,4,Blue Line,Pacific Coast Hwy Station,80106,Stop: Pacific Coast Hwy Station\nStop No: 8010...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
4,-118.18983,33.80708,5,Blue Line,Willow Street Station,80107,Stop: Willow Street Station\nStop No: 80107\nB...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro


In [12]:
# concat the two tables
frames = [permit_data_transformed[['type','address_full', 'permit_sub_type', 'lat', 'long']].rename(columns=\
            {'type': 'type', 'address_full':  'name', 
             'permit_sub_type': 'sub_type', 'lat': 'latitude', 'long': 'longitude'}),
          metro_lines[['type', 'Station', 'MetroLine','Y', 'X']].rename(columns=\
            {'type': 'type', 'Station': 'name', 'MetroLine': 'sub_type',
            'Y': 'latitude', 'X': 'longitude'})]

permits_metro_full = pd.concat(frames)

In [13]:
permits_metro_full.head()

,type,name,sub_type,latitude,longitude
0,Permit,"5826 N MELVIN AVE LOS ANGELES, CA 91356",1 or 2 Family Dwelling,,
1,Permit,"3133 W LACLEDE AVE LOS ANGELES, CA 90039",1 or 2 Family Dwelling,,
2,Permit,"20318 W WINDSOR LANE LOS ANGELES, CA 91326",1 or 2 Family Dwelling,,
3,Permit,"3715 S WASATCH AVE LOS ANGELES, CA 90066",1 or 2 Family Dwelling,,
4,Permit,"3564 S GRAND VIEW BLVD LOS ANGELES, CA 90066",1 or 2 Family Dwelling,,


In [14]:
permits_metro_full.tail()


,type,name,sub_type,latitude,longitude
99,Metro,Westlake / MacArthur Park Station,Purple Line,34.05637,-118.27488
100,Metro,7th St / Metro Center Station - Metro Red & Pu...,Purple Line,34.04863,-118.25868
101,Metro,Pershing Square Station,Purple Line,34.04932,-118.25126
102,Metro,Civic Center / Grand Park Station,Purple Line,34.05490,-118.24606
103,Metro,Union Station - Metro Red & Purple Lines,Purple Line,34.05620,-118.23425


In [15]:
metro_lines.head()

,X,Y,OBJECTID,MetroLine,Station,StopNumber,TOOLTIP,NLA_URL,type
0,-118.19292,33.76807,1,Blue Line,Downtown Long Beach Station,80101,Stop: Downtown Long Beach Station\nStop No: 80...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
1,-118.19370,33.77226,2,Blue Line,Pacific Ave Station,80102,Stop: Pacific Ave Station\nStop No: 80102\nBlu...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
2,-118.18938,33.78183,3,Blue Line,Anaheim Street Station,80105,Stop: Anaheim Street Station\nStop No: 80105\n...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
3,-118.18938,33.78909,4,Blue Line,Pacific Coast Hwy Station,80106,Stop: Pacific Coast Hwy Station\nStop No: 8010...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro
4,-118.18983,33.80708,5,Blue Line,Willow Street Station,80107,Stop: Willow Street Station\nStop No: 80107\nB...,http://www.metro.net/riding/maps/blue-line/?nl...,Metro


In [16]:
# export to excel
permits_metro_full.to_excel(r'LA_permits_metro_full.xlsx')

In [17]:
# create df of each metro stop and its address, lat/lon, city name

metro_addresses = pd.DataFrame()
metro_addresses['latitude'] = ''
metro_addresses['longitude'] = ''
metro_addresses['formatted_address'] = ''
metro_addresses['city_name'] = ''
metro_addresses['metro_line'] = ''
metro_addresses['station_name'] = ''

for x in range(len(metro_lines)):
    metro_addresses.loc[x, 'latitude'] = metro_lines['Y'][x]
    metro_addresses.loc[x, 'longitude'] = metro_lines['X'][x]
    metro_addresses.loc[x, 'metro_line'] = metro_lines['MetroLine'][x]
    metro_addresses.loc[x, 'station_name'] = metro_lines['Station'][x]
    lat_long = (float(metro_lines['Y'][x]), float(metro_lines['X'][x]))
    reverse_geocode_result = gmaps.reverse_geocode(lat_long)
    metro_addresses.loc[x, 'formatted_address'] = reverse_geocode_result[0]['formatted_address']
    address_component = reverse_geocode_result[0]['address_components']
    for y in address_component:
        if y['types'] == ['locality', 'political']:
            metro_addresses.loc[x, 'city_name'] = y['long_name']
metro_addresses.head()

In [18]:
# check for nulls
metro_addresses[metro_addresses['city_name'].isnull()]

In [20]:
# look up info for nulls
print(gmaps.reverse_geocode((33.848222,-118.211017))[0]['address_components'])
print(gmaps.reverse_geocode((33.928683,-118.291733))[0]['address_components'])

In [22]:
# manually set address to appropriate city
metro_addresses.loc[metro_addresses['station_name'] == 'Del Amo Station', 'city_name'] = 'Los Angeles'
metro_addresses.loc[metro_addresses['station_name'] == 'Vermont / Athens Station', 'city_name'] = 'Los Angeles'
metro_addresses.head(10)

In [23]:
metro_addresses.to_csv(r'metro_addresses.csv')